In [11]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

# from datetime import *
# today = date.today()
# today = today.strftime("%m.%d.20%y")
# today

time = "7pm_"
import datetime
today = date.today()
today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'04.01.2022'

In [10]:
upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\Template\FD_Upload_PlayerPool_7pm_04.01.2022.csv'
                     , usecols = [0,1,2], 
                     error_bad_lines=False, engine ='python')
upload = upload.dropna()
upload

,entry_id,contest_id,contest_name
0,2.800816e+09,73826-255951169,NBA Beat The Score: 270 ($800 Guaranteed)
1,2.800816e+09,73826-255951014,$30K Fri NBA Pick 'n Roll ($3K to 1st)
2,2.800817e+09,73826-255951004,$5K Fri NBA Dribbler (Single Entry)
3,2.800817e+09,73826-255951012,$4K Fri NBA Block (Single Entry)
4,2.800817e+09,73826-255951019,$2K Fri NBA EPE Shot (No Experienced Players)
5,2.800817e+09,73826-255956042,NBA Mini Swat (EPE)
6,2.801234e+09,73826-255966380,$250 NBA Small Swat


In [27]:
## FanDuel Player Pool



contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

# fd = glob.glob('*-players-list.csv')[0]

fd_upload = glob.glob('*upload-template.csv')[0]
df = pd.read_csv(fd_upload, header=6, usecols=[13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,30])
df.to_csv("FDSalaries_"+ contest + time + str(today) + ".csv", index=False)
# os.rename(fd, "FDSalaries_"+ contest + time + str(today) + ".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + time + str(today) + ".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + time + str(today) + ".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + time + str(today) + ".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + time + str(today) + ".xlsx", index=False)
os.remove("FDSalaries_"+ contest + time + str(today) + ".csv")

## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

fanduel_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\Template"

fd_upload = glob.glob('*upload-template.csv')[0]
os.rename(fd_upload, "FD_Upload_"+ contest + time + str(today) + ".csv")

shutil.move(downloads + "/"+ "FD_Upload_"+ contest + time + str(today) + ".csv",
           fanduel_upload + "/"+ "FD_Upload_"+ contest + time + str(today) + ".csv")

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\Upload\\FanDuel\\Template/FD_Upload_PlayerPool_9pm_04.01.2022.csv'

In [28]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+ time + str(today) +".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [12]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + time + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + time + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id', 'Salary']], 
              left_on='Player', 
              right_on='Nickname')


# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

# Concat the list of positions
df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
# df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Name + ID'] = df['Id'] + ":" + df['Player']

df.to_csv("FD_final_PlayerPool_ETR_"+ time + str(today) +".csv", index=False)

df = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ time + str(today) + ".csv")


## Unnests FD Positions into each Row (explode in Python) ##

# Update Salary to INT 
# df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

# Update FD Position to a string
df['FD Position'] = df['FD Position'].astype(str)
#Delimit by '/'
df['FD Position'] = df['FD Position'].str.split('/')

# Unnest columns by FD Position
df = df.explode('FD Position')

# Create Position + ID as a lookup value
df['Position + Id'] = df['FD Position'] + "_" + df['Id']

df.head()

df.to_csv("FD_final_PlayerPool_ETR_"+ time + str(today) +".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+ time + str(today) +".xlsx")

# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG = 2
- TotalPosition_SG = 2
- TotalPosition_SF = 2
- TotalPosition_PF =2 
- TotalPosition_C =1


In [13]:
## Optimizer

df = pd.read_csv("FD_final_PlayerPool_ETR_"+ time + str(today) +".csv", index_col=['Id', 'FD Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['Salary'].to_dict()
values = df['FD Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 60000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

73826-67026 SF
73826-55062 C
73826-157808 SG
73826-110353 PF
73826-84703 SG
73826-42156 PF
73826-145313 SF
73826-145536 PG
73826-157812 PG


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [14]:
# Transpose dataframe from column to row

new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]

# Create a lookup variable for [Position + Id]
test['lookup'] = test[1] + "_" + test[0]


# Merge PuLP output on [Position + Id] to pull in Player Name
df = pd.read_csv("FD_final_PlayerPool_ETR_"+ time + str(today) +".csv")
upload = pd.merge(test,
                 df[['Player', 'Position + Id', 'Name + ID']],
                 left_on = 'lookup',
                 right_on = 'Position + Id')


# Drop all columns not used in upload
upload2 = upload.drop([0, 'lookup', 'Player', 'Position + Id'], axis = 1)


# Transpose data output from vertical to horizontal
upload3 = upload2.transpose()


# Rename columns to first row (position values)
upload4 = upload3.rename(columns=upload3.iloc[0])


# Drop duplicate first row
upload5 = upload4.drop([1])


# Re-order the columns to the Spositions
upload6 = upload5[['PG',
                   'SG',
                   'SF',
                   'PF',
                   'C']]


# Save as a csv
upload6.to_csv("fd_result_" + time + str(today) + ".csv", index=None)

# os.system("fd_result_" + time + str(today) + ".csv")

upload6

C:\Users\vchang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PG,PG,SG,SG,SF,SF,PF,PF,C
Name + ID,73826-145536:Theo Maledon,73826-157812:Davion Mitchell,73826-157808:Cade Cunningham,73826-84703:De'Anthony Melton,73826-67026:Jaylen Brown,73826-145313:Jordan Nwora,73826-110353:Brandon Clarke,73826-42156:Bobby Portis,73826-55062:Nikola Jokic


In [15]:
upload6 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\fd_result_' + time + str(today) + ".csv")
upload6.head()


upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\Template\FD_Upload_PlayerPool_' + time + str(today) + ".csv"
                     , usecols = [0,1,2], 
                     error_bad_lines=False, engine ='python')
upload = upload.dropna()
upload = upload[['entry_id', 'contest_id', 'contest_name']]

# Drops rows that have NaN
upload2 = upload.dropna()
# creates value for row counts in the upload file
repeat_rows = len(upload2.index)

# duplicates lineups down rows based on number of contests (length of rows)
# copies columns from old data frame onto new dataframe
df3 = pd.DataFrame(np.repeat(upload6.values, repeat_rows, axis=0))
df3.columns = upload6.columns

# joins the two data frames
os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel')
result = pd.concat([upload2, df3], axis=1)


test = result.rename(columns={"PG.1":"PG", "SG.1":"SG", "SF.1":"SF", "PF.1":"PF"})
result.to_csv("FD_Final_Upload_Base_"+ time + str(today) + ".csv", index=False)
os.system("FD_Final_Upload_Base_"+ time + str(today) + ".csv")

0